# **Proyecto 1 - Etapa I**

Integrantes:
* Andrés Francisco Borda - 201729184
* Juan Pablo Lora Hernández - 202012524
* Gabriela Vargas Rojas - 202013830


# 1. Entendimiento de los datos

In [1]:
#Instalaciones

#Libreria para remplazar numeros por palabras
!pip install num2words
#Instalar  pandas profiler
!pip install pandas-profiling==2.7.1
#deteccion de lenguaje para eliminar entradas que no esten en español
!pip install langdetect
#Procesamientno de lenguaje natural en español
!pip install stanza

You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement pandas-profiling==2.7.1 (from versions: 3.0.0, 3.1.0, 3.2.0, 3.3.0, 3.4.0, 3.5.0, 3.6.0, 3.6.1, 3.6.2, 3.6.3, 3.6.4, 3.6.5, 3.6.6)
ERROR: No matching distribution found for pandas-profiling==2.7.1
You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.


  Using cached stanza-1.6.1-py3-none-any.whl (881 kB)
  Using cached numpy-1.26.0-cp39-cp39-win_amd64.whl (15.8 MB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached emoji-2.8.0-py2.py3-none-any.whl (358 kB)
  Using cached torch-2.1.0-cp39-cp39-win_amd64.whl (192.2 MB)
  Using cached protobuf-4.24.4-cp39-cp39-win_amd64.whl (430 kB)
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached fsspec-2023.9.2-py3-none-any.whl (173 kB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached charset_normalizer-3.3.0-cp39-cp39-win_amd64.whl (98 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Python39\\share'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'C:\Python39\python.exe -m pip install --upgrade pip' command.


In [2]:
#imports para procesamiento de texto

#tokenizacion y lematizacion
import stanza
#Para integrar pasos de la limpieza adicionales
from stanza.pipeline.processor import Processor, register_processor
#paquete español
stanza.download('es')

#Para manejo de numeros, singluares, plurarles en lenguaje
from num2words import num2words
#Deteccion de lenguaje
from langdetect import detect
# librería Natural Language Toolkit, usada para trabajar con textos
import nltk
# Punkt permite separar un texto en frases.
nltk.download('stopwords')
from nltk.corpus import stopwords

#Operaciones con expresiones regulares y unicode
import re, string, unicodedata

ModuleNotFoundError: No module named 'stanza'

In [ ]:
#Imports generales para analisis de datos y ML
import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport
import statistics

import sklearn
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# Entendimiento de los datos

In [ ]:
# Lectura de los datos.
df_ODS = pd.read_excel("cat_6716.xlsx")


df_ODS.head()


In [ ]:
df_ODS.dtypes

#Imprimimos reporte de pandas Profiler

In [ ]:
mensajes = df_ODS.copy()
#Longitud del comentario (numero de caracteres)
mensajes['Conteo'] = [len(x) for x in mensajes['Textos_espanol']]
#Palabra(s) mas frecuente(s)
mensajes['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in mensajes['Textos_espanol']]
#Longitud de la palabra mas larga del comentario
mensajes['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in mensajes['Textos_espanol']]
#Longitud de la palabra mas corta del comentario
mensajes['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in mensajes['Textos_espanol']]

ProfileReport(mensajes)


In [ ]:
#Conteo de entradas nulas
df_ODS.isnull().sum()

In [ ]:
#verificamos entradas en otros idiomas
df_ODS[df_ODS.Textos_espanol.map(detect)  != 'es']

In [ ]:
#Revisamos los registros duplicados
df_ODS.duplicated().sum()

# 2. Preparacion de los datos

In [ ]:
#Eliminamos Duplicados
df_ODS = df_ODS.drop_duplicates()
#Eliminamos entradas en otros idiomas
df_ODS = df_ODS.drop(df_ODS[df_ODS.Textos_espanol.map(detect)  != 'es'].index)

In [ ]:
 #verificamos cuantos datos quedan
 df_ODS.shape

## 2.1 Limpieza, tokenizacion y lemmatizacion

In [ ]:
#Remplaza los numeros por su representacion en palabras
def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = num2words(word, lang='es')
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words
#Remueve todo caracter no latino (conserva espacios y numeros)
def remove_nonlatin(words):
  new_words = []
  for word in words:
    new_word = ''
    for ch in word:
      if unicodedata.name(ch).startswith(('LATIN', 'DIGIT', 'SPACE')):
        new_word += ch
    new_words.append(new_word)
  return new_words

#Remueve palabras comunes que no aportan informacion
def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    s = set(stopwords.words('spanish'))
    for word in words:
        if word not in s:
            new_words.append(word)
    return new_words

#Remueve puntuacion
def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = ''
    for word in words:
            new_words += re.sub(r'[^\w\s]', ' ', word)
    return new_words

 #Procesamiento de cada review usando stanza
def tokenLemma():
  df_ODS['words'] = df_ODS['Textos_espanol'].apply(remove_punctuation)
  #Creamos un pipeline para tokenizacion y lematizacion
  nlp = stanza.Pipeline('es', processors = 'tokenize,mwt,pos,lemma', use_gpu=True)
  in_docs = [stanza.Document([], text=d) for d in df_ODS.words]
  return nlp(in_docs)

#Funcion secundaria para procesar cada token
def procesamientoPalabras(words):
    words = remove_nonlatin(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

#Funcion principal para el pre-procesamiento
def preprocessing():
  out_docs = tokenLemma()
  palabras = []

  for doc in out_docs:
    reviewAct = []
    for sentence in doc.sentences:
      for word in sentence.words:
        if(word.pos != 'PUNCT' and word.pos != 'SYM'):
          reviewAct.append(word.lemma.lower())
    palabras.append(reviewAct)

  df_ODS['words'] = palabras
  df_ODS['words'] = df_ODS['words'].apply(procesamientoPalabras)

In [ ]:
#Descomentar el processing si se quiere revisar, si no el resultado con los datos procesados esta en el archivo df_procesado
#preprocessing()


In [ ]:
# Cargamos los datos procesados

#Guardamos los datos procesados. Descomentar si se quiere crear el archivo despues de procesar los datos.
#df_ODS.to_csv('df_procesado.csv', index=False, sep= ";")

#Leemos los datos procesados
df_ODS_Pre = pd.read_csv("df_procesado.csv", sep = ";", encoding='utf-8')
import ast
df_ODS_Pre['words'] = df_ODS_Pre['words'].apply(ast.literal_eval)



In [ ]:
#Visualizar los cambios
df_ODS_Pre.head()

In [ ]:
#Convertimos los tokens nuevamente en strings
df_ODS_Pre['words'] = df_ODS_Pre['words'].apply(lambda x: ' '.join(map(str, x)))
df_ODS_Pre

### 2.1.1 Verificamos los datos obtenidos

In [ ]:
mensajes = df_ODS_Pre.copy()
#Longitud del comentario (numero de caracteres)
mensajes['Conteo'] = [len(x) for x in mensajes['words']]
#Palabra(s) mas frecuente(s)
mensajes['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in mensajes['words']]
#Longitud de la palabra mas larga del comentario
mensajes['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in mensajes['words']]
#Longitu de la palabra mas corta del comentario
mensajes['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in mensajes['words']]

ProfileReport(mensajes)

## 2.2 Separacion de los datos y vectorización

In [ ]:
#Separación de los datos en conjunto de test y train
df_ODS_Pre = df_ODS_Pre.drop('Textos_espanol', axis = 1)
df_train, df_test = sklearn.model_selection.train_test_split(df_ODS_Pre, test_size=0.2, random_state=0)

X_train = df_train['words']
y_train = df_train['sdg']

X_test = df_test['words']
y_test = df_test['sdg']

In [ ]:
#Vectorizar los datos con Tfid
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train)
test_vectors = vectorizer.transform(X_test)
print(train_vectors.shape, test_vectors.shape)

# 3. Modelado y evaluacion

## Random Forest - Gabriela Vargas

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clasif = RandomForestClassifier(random_state=0)
clasif.fit(train_vectors, y_train)
ytest_pred = clasif.predict(test_vectors)

clasif = RandomForestClassifier(random_state=0)
clasif.fit(train_vectors, y_train)
predicted = clasif.predict(test_vectors)
class_report = classification_report(y_test, predicted)
print("\nClassification Report")
print(class_report)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

#Se fijan 5 particiones teniendo en cuenta los datos que estamos manejando.
particiones = KFold(n_splits=5, shuffle=True, random_state = 0)

# Establecemos el espacio de búsqueda para los hiperparámetros en especial los que nos permiten controlar la complejidad y el rendimiento del modelo
param_grid = {'criterion':['gini', 'entropy'],'min_samples_split':[2, 4, 6],'max_features':[10, 100, 1000]}

forest = RandomForestClassifier(random_state=0)
modelo = GridSearchCV(forest, param_grid, cv=particiones, refit = True, scoring = 'accuracy')
modelo.fit(train_vectors, y_train)

print(modelo.best_params_)  #MMejor modelo escogido

clasif = modelo.best_estimator_
predicted = clasif.predict(test_vectors)
class_report = classification_report(y_test, predicted)
print("\nClassification Report")
print(class_report)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predicted)

Teniendo en cuenta el accuaracy score, nos pudimos dar cuenta que el Random Forest está realizando buenas predicciones con los datos que tenemos, lo cual nos puede indicar que este modelo nos permite relacionar de manera automática un texto según los ODS de manera casi precisa.